1. 경로 읽어들이기 읽어들이기
    - ```glob``` 활용
    - croparea.properties, 이미지들의 경로가 포함되어 있음
    - 일단 이 경로를 그대로 함수에 전달
2. ```cropping``` 함수
    - input : 경로가 담긴 list형태로 입력
<br>   
    - 다음과 같은 작업을 수행
        1. ```crop_area.properties``` 경로 저장  
        2. ```crop_area.properties``` 파일을 읽고 반복문 실행
        3. crop영역에 대한 정보를 저정
            - x, y, w, h 정보
            - ```img[y, y+h, x, x+w]```
        4. crop_area에서 명시해주는 파일명을 이용, img 경로를 활용해서 Image 열기
        5. 다음과 같은 형식으로 Image를 crop 시도
            1. Pillow로 이미지 읽기
            2. RGBA 형식을 RGB로 변환
            3. 지정한 범위로 이미지를 크롭
            4. 새로운 경로에 이미지를 저장
        6. cropping을 하지 않은 온전한 이미지를 저장 디렉토리에 복사

In [ ]:
import os
import shutil
from glob import glob
from PIL import Image

import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
# 물냉면을 예로 들었을 때
class_paths = glob('/content/drive/My Drive/data/kfood/물냉면/*')

In [ ]:
def ret_img_path(all_paths, fname) :
    for path in all_paths :
        if fname in path:
            return path

In [ ]:
class_paths[0].split('/')[-1]

'Img_052_0593.jpg'

In [ ]:
def cropping(class_paths):
    # 결과물은 새로운 폴더에 저장
    # 클래스명을 활용해서 cropped_'class'폴더 생성
    class_name = class_paths[0].split('/')[-2]
    save_dir_path = '/content/drive/My Drive/data/kfood/cropped_' + class_name + '/'
    if not os.path.exists(save_dir_path):
        os.makedirs(save_dir_path)

    # crop_area path 저장
    for path in class_paths :
        if 'crop_area' in path :
            crop_path = path
            break

    # crop_area파일 읽기
    f = open(crop_path, 'r')
    data = f.read()
    f.close()

    crop_infos = data.split('\n')
    cropped_fname = []

    # crop에 대한 정보가 있는 파일들은 crop을 하고,
    # 앞에서 새로 만든 cropped_classname 디렉터리에 저장
    print("crop 시작")
    for crop_info in tqdm(crop_infos) :

        fname = crop_info.split('=')[0]
        try :
            x, y, w, h = list(map(int, crop_info.split('=')[-1].split(',')))
        except :
            pass

        cropped_fname.append(fname)
        
        img_path = ret_img_path(class_paths, fname)
        image = Image.open(img_path)
        image = image.convert("RGB")
        cropped_img = image.crop((x, y, x+w, y+h))
        cropped_img.save(save_dir_path + 'cropped_' + fname + '.jpg')

    print("crop 끝")
    
    print("\n\ncrop을 하지 않은 이미지를 복사 중입니다.")
    print("잠시만 기다려 주세요")
    # crop이 필요없는 image들의 경우..
    # crop 경로에 똑같이 복사를 해준다
    for path in tqdm(class_paths):
        fname = path.split('/')[-1].split('.')[0]
        if fname not in cropped_fname and 'csv' not in path and 'properties' not in path:
            shutil.copy(path, save_dir_path)

In [ ]:
mulnang = glob('/content/drive/My Drive/data/kfood/물냉면/*')
len(mulnang)

1002

In [ ]:
cropping(mulnang)

crop 시작



crop 끝


crop을 하지 않은 이미지를 복사 중입니다.
잠시만 기다려 주세요


In [ ]:
len(os.listdir('/content/drive/My Drive/data/kfood/cropped_물냉면/'))

1001